players__dataset_creation code for past seasons

Select the season in season variable, and repeat the code if needed

In [1]:
season = '2223'

In [2]:
import pandas as pd

In [3]:
rcsv = pd.read_csv('fbref_data/season' + season + '/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/season' + season + '/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [4]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

players

player           team
0       James Abankwah        Udinese
1    Oliver Abildgaard  Hellas Verona
2        Tammy Abraham           Roma
3     Christian Acella      Cremonese
4     Francesco Acerbi          Inter
..                 ...            ...
647        Martin Turk      Sampdoria
648       Samir Ujkani         Empoli
649  Guglielmo Vicario         Empoli
650        Jeroen Zoet         Spezia
651        Petar Zovko         Spezia

[652 rows x 2 columns]

In [5]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [6]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [7]:
print(players[['player', 'surname']].to_string())

                         player           surname
0                James Abankwah          Abankwah
1             Oliver Abildgaard        Abildgaard
2                 Tammy Abraham           Abraham
3              Christian Acella            Acella
4              Francesco Acerbi            Acerbi
5                   Yacine Adli              Adli
6              Michel Aebischer         Aebischer
7              Felix Afena-Gyan        Afena-Gyan
8                 Kevin Agudelo           Agudelo
9                      Ola Aina              Aina
10                Emanuel Aiwum             Aiwum
11       Jean-Daniel Akpa-Akpro        Akpa-Akpro
12                 Luis Alberto           Alberto
13     Agustín Álvarez Martínez          Martinez
14                 Kelvin Amian             Amian
15                 Bruno Amione            Amione
16                 Bruno Amione            Amione
17                 Ethan Ampadu            Ampadu
18               Sofyan Amrabat           Amrabat


In [8]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix


Ostigard
Kim
Hojlund
Gytkjaer
Maehle
Kjaer
Djuricic
Djuric
Cabral
Alvarez


FROM           TO           TEAM
0      stigard     Ostigard         Napoli
1      Min-jae          Kim         Napoli
2       Hjlund      Hojlund       Atalanta
3       Gytkjr     Gytkjaer          Monza
4       Carlos      Augusto          Inter
5         Mhle       Maehle       Atalanta
6          Kjr        Kjaer          Milan
7       uricic     Djuricic      Sampdoria
8         uric       Djuric  Hellas Verona
9       Arthur       Cabral     Fiorentina
10    Martinez      Alvarez       Sassuolo
11  Gumundsson  Gudmundsson          Genoa
12  Kristensen       Nissen           Roma

In [9]:
fc_data = pd.read_excel('fantacalcio/season' + season + '/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players.loc[i, 'surname'] = spl[-2]
        fc_players.loc[i, 'initial'] = spl[-1][0]
    else:
        fc_players.loc[i, 'surname'] = spl[-1]
        fc_players.loc[i, 'initial'] = ''
    
fc_players



id  r                 name       team    surname initial
0     572  P                Meret     Napoli      Meret        
1    2814  P             Provedel      Lazio   Provedel        
2    4964  P              Vicario     Empoli    Vicario        
3     453  P             Szczesny   Juventus   Szczesny        
4    2134  P              Falcone      Lecce    Falcone        
..    ... ..                  ...        ...        ...     ...
538  5512  A              De Luca  Sampdoria       Luca        
539  5837  A  Voelkerling Persson      Lecce    Persson        
540  6113  A            Montevago  Sampdoria  Montevago        
541  6143  A              Krollis     Spezia    Krollis        
542  6160  A              Vivaldo    Udinese    Vivaldo        

[543 rows x 6 columns]

In [10]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players.loc[i, 'fb_ID']  = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):              
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                if((fc_players['r'][i] == 'P') == (j >= keepers_ID)): # check wether they're a goalkeeper for both FBREF and Fantacalcio
                    fc_players.loc[i, 'fb_ID'] = j

NameError: name 'keepers_ID' is not defined

In [ ]:
for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        print(fc_players['name'][i])

Lamanna
Berisha
Padelli
Pinsoglio
Ravaglia F.
Brancolini
Bleve
Berardi A.
Gemello
Boer
Adamonis
Marfella
Piana
Bagnolini
Sorrentino A.
Ciezkowski
Saro
Vasquez D.
Aiwu
Paletta
Fares
Amey
Guarino
Machin
Akpa Akpro
Galdames
Darboe
Urbanski
Trimboli
Samek
Degli Innocenti
Faticanti
Oddei
Kaio Jorge
Vivaldo


In [ ]:
fc_players

id  r                 name       team    surname initial  fb_ID
0     572  P                Meret     Napoli      Meret            620
1    2814  P             Provedel      Lazio   Provedel            632
2    4964  P              Vicario     Empoli    Vicario            649
3     453  P             Szczesny   Juventus   Szczesny            644
4    2134  P              Falcone      Lecce    Falcone            612
..    ... ..                  ...        ...        ...     ...    ...
538  5512  A              De Luca  Sampdoria       Luca            151
539  5837  A  Voelkerling Persson      Lecce    Persson            565
540  6113  A            Montevago  Sampdoria  Montevago            365
541  6143  A              Krollis     Spezia    Krollis            288
542  6160  A              Vivaldo    Udinese    Vivaldo             -1

[543 rows x 7 columns]

In [ ]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            #fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            fc_players.loc[i, columns_to_copy[j]] = outfield_players.loc[fc_players['fb_ID'][i], columns_to_copy[j]]
            

C:\Users\Peppe\AppData\Local\Temp\ipykernel_3484\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\Peppe\AppData\Local\Temp\ipykernel_3484\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\Peppe\AppData\Local\Temp\ipykernel_3484\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [ ]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [ ]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
        if(fc_players['fb_ID'][i] - delta_k >= 0):
            for j in range(columns_to_copy.shape[0]):          
                #fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]
                fc_players.loc[i, columns_to_copy[j]] = keeper_players.loc[fc_players['fb_ID'][i] - delta_k, columns_to_copy[j]]

C:\Users\Peppe\AppData\Local\Temp\ipykernel_3484\3175819694.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\Peppe\AppData\Local\Temp\ipykernel_3484\3175819694.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\Peppe\AppData\Local\Temp\ipykernel_3484\3175819694.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [ ]:
keeper_players

player nationality position           team  age  \
0              Emil Audero      it ITA       GK      Sampdoria   25   
1          Francesco Bardi      it ITA       GK        Bologna   30   
2        Marco Carnesecchi      it ITA       GK      Cremonese   22   
3       Michele Cerofolini      it ITA       GK     Fiorentina   23   
4          Andrea Consigli      it ITA       GK       Sassuolo   35   
5              Alex Cordaz      it ITA       GK          Inter   39   
6           Alessio Cragno      it ITA       GK          Monza   28   
7      Michele Di Gregorio      it ITA       GK          Monza   25   
8     Bartłomiej Drągowski      pl POL       GK         Spezia   24   
9        Wladimiro Falcone      it ITA       GK          Lecce   27   
10       Vincenzo Fiorillo      it ITA       GK    Salernitana   32   
11       Pierluigi Gollini      it ITA       GK         Napoli   27   
12       Pierluigi Gollini      it ITA       GK     Fiorentina   27   
13        Samir Handanović      si SVN       GK          Inter   38   
14            Mike Maignan      fr FRA       GK          Milan   27   
15      Federico Marchetti      it ITA       GK         Spezia   39   
16          Luís Maximiano      pt POR       GK          Lazio   23   
17              Alex Meret      it ITA       GK         Napoli   25   
18  Vanja Milinković-Savić      rs SRB       GK         Torino   25   
19         Antonio Mirante      it ITA       GK          Milan   39   
20         Lorenzo Montipò      it ITA       GK  Hellas Verona   26   
21              Juan Musso      ar ARG       GK       Atalanta   28   
22         Guillermo Ochoa      mx MEX       GK    Salernitana   37   
23             André Onana      cm CMR       GK          Inter   26   
24            Rui Patrício      pt POR       GK           Roma   34   
25         Gianluca Pegolo      it ITA       GK       Sassuolo   41   
26          Simone Perilli      it ITA       GK  Hellas Verona   27   
27            Mattia Perin      it ITA       GK       Juventus   29   
28         Samuele Perisan      it ITA       GK         Empoli   24   
29           Ivan Provedel      it ITA       GK          Lazio   28   
30              Ionuț Radu      ro ROU       GK      Cremonese   25   
31         Nicola Ravaglia      it ITA       GK      Sampdoria   33   
32         Francesco Rossi      it ITA       GK       Atalanta   31   
33        Alessandro Russo      it ITA       GK       Sassuolo   21   
34         Mouhamadou Sarr      sn SEN       GK      Cremonese   25   
35              Luigi Sepe      it ITA       GK    Salernitana   31   
36         Marco Silvestri      it ITA       GK        Udinese   31   
37        Salvatore Sirigu      it ITA       GK     Fiorentina   35   
38        Łukasz Skorupski      pl POL       GK        Bologna   31   
39        Marco Sportiello      it ITA       GK       Atalanta   30   
40             Mile Svilar      rs SRB       GK           Roma   22   
41       Wojciech Szczęsny      pl POL       GK       Juventus   32   
42      Ciprian Tătărușanu      ro ROU       GK          Milan   36   
43      Pietro Terracciano      it ITA       GK     Fiorentina   32   
44             Martin Turk      si SVN       GK      Sampdoria   18   
45            Samir Ujkani      xk KVX       GK         Empoli   34   
46       Guglielmo Vicario      it ITA       GK         Empoli   25   
47             Jeroen Zoet      nl NED       GK         Spezia   31   
48             Petar Zovko      ba BIH       GK         Spezia   20   

    birth_year  gk_games  gk_games_starts  gk_minutes  gk_goals_against  ...  \
0         1997      25.0             25.0      2250.0              39.0  ...   
1         1992       1.0              1.0        90.0               0.0  ...   
2         2000      27.0             27.0      2430.0              47.0  ...   
3         1999       5.0              5.0       450.0               3.0  ...   
4         1987      35.0             35.0      3150.0              55

In [ ]:
fc_players

id  r                 name       team    surname initial  fb_ID  age  \
0     572  P                Meret     Napoli      Meret            620   25   
1    2814  P             Provedel      Lazio   Provedel            632   28   
2    4964  P              Vicario     Empoli    Vicario            649   25   
3     453  P             Szczesny   Juventus   Szczesny            644   32   
4    2134  P              Falcone      Lecce    Falcone            612   27   
..    ... ..                  ...        ...        ...     ...    ...  ...   
538  5512  A              De Luca  Sampdoria       Luca            151   24   
539  5837  A  Voelkerling Persson      Lecce    Persson            565   19   
540  6113  A            Montevago  Sampdoria  Montevago            365   19   
541  6143  A              Krollis     Spezia    Krollis            288   20   
542  6160  A              Vivaldo    Udinese    Vivaldo             -1    0   

     birth_year  games  ...  gk_passes_length_avg  gk_goal_kicks  \
0          1997      0  ...                  26.1            198   
1          1994      0  ...                  32.2            198   
2          1996      0  ...                  33.8            170   
3          1990      0  ...                  33.5            153   
4          1995      0  ...                  42.1            282   
..          ...    ...  ...                   ...            ...   
538        1998      2  ...                   0.0              0   
539        2003      9  ...                   0.0              0   
540        2003      6  ...                   0.0              0   
541        2001      4  ...                   0.0              0   
542           0      0  ...                   0.0              0   

     gk_pct_goal_kicks_launched  gk_goal_kick_length_avg  gk_crosses  \
0                          20.2                     26.9         335   
1                          33.3                     33.2         493   
2                          50.0                     42.9         606   
3                          43.1                     38.3         389   
4                          79.1                     53.5         534   
..                          ...                      ...         ...   
538                         0.0                      0.0           0   
539                         0.0                      0.0           0   
540                         0.0                      0.0           0   
541                         0.0                      0.0           0   
542                         0.0                      0.0           0   

     gk_crosses_stopped  gk_crosses_stopped_pct  \
0                    11                     3.3   
1                    21                     4.3   
2                    34                     5.6   
3                    11                     2.8   
4                    23                     4.3   
..                  ...                     ...   
538                   0                     0.0   
539                   0                     0.0   
540                   0                     0.0   
541                   0                     0.0   
542                   0                     0.0   

     gk_def_actions_outside_pen_area  gk_def_actions_outside_pen_area_per90  \
0                                 39                                   1.15   
1                                 55                                   1.45   
2                                 22                                   0.71   
3                                 21                                   0.76   
4                                 42                                   1.11   
..                               ...                                    ...   
538                                0                                   0.00   
539                                0                                   0.00   
540                                0                

In [ ]:
import numpy as np

votes = pd.read_excel('mid_outputs/season' + season + '/players_votes.xlsx', index_col = 0)

mean_def = 6
std_def = 0.58

mean_def_P = 6.22
std_def_P = 0.43


min_votes = 6


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

vote_avg  vote_std
0    6.176471  0.400043
1    6.263158  0.409378
2    6.403226  0.447562
3    6.107143  0.309295
4    6.250000  0.547122
..        ...       ...
538  5.871240  0.394770
539  6.132748  0.474211
540  5.722482  0.229188
541  6.357731  0.405367
542  6.006617  0.274006

[543 rows x 2 columns]

In [ ]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

id  r                 name       team    surname initial  fb_ID  age  \
0     572  P                Meret     Napoli      Meret            620   25   
1    2814  P             Provedel      Lazio   Provedel            632   28   
2    4964  P              Vicario     Empoli    Vicario            649   25   
3     453  P             Szczesny   Juventus   Szczesny            644   32   
4    2134  P              Falcone      Lecce    Falcone            612   27   
..    ... ..                  ...        ...        ...     ...    ...  ...   
538  5512  A              De Luca  Sampdoria       Luca            151   24   
539  5837  A  Voelkerling Persson      Lecce    Persson            565   19   
540  6113  A            Montevago  Sampdoria  Montevago            365   19   
541  6143  A              Krollis     Spezia    Krollis            288   20   
542  6160  A              Vivaldo    Udinese    Vivaldo             -1    0   

     birth_year  games  ...  gk_pct_goal_kicks_launched  \
0          1997      0  ...                        20.2   
1          1994      0  ...                        33.3   
2          1996      0  ...                        50.0   
3          1990      0  ...                        43.1   
4          1995      0  ...                        79.1   
..          ...    ...  ...                         ...   
538        1998      2  ...                         0.0   
539        2003      9  ...                         0.0   
540        2003      6  ...                         0.0   
541        2001      4  ...                         0.0   
542           0      0  ...                         0.0   

     gk_goal_kick_length_avg  gk_crosses  gk_crosses_stopped  \
0                       26.9         335                  11   
1                       33.2         493                  21   
2                       42.9         606                  34   
3                       38.3         389                  11   
4                       53.5         534                  23   
..                       ...         ...                 ...   
538                      0.0           0                   0   
539                      0.0           0                   0   
540                      0.0           0                   0   
541                      0.0           0                   0   
542                      0.0           0                   0   

     gk_crosses_stopped_pct  gk_def_actions_outside_pen_area  \
0                       3.3                               39   
1                       4.3                               55   
2                       5.6                               22   
3                       2.8                               21   
4                       4.3                               42   
..                      ...                              ...   
538                     0.0                                0   
539                     0.0                                0   
540                     0.0                                0   
541                     0.0                                0   
542                     0.0                                0   

     gk_def_actions_outside_pen_area_per90  gk_avg_distance_def_actions  \
0                                     1.15                         17.0   
1                                     1.45                         16.5   
2                                     0.71                         11.5   
3                                     0.76                         14.7   
4                                     1.11                         14.1   
..                                     ...                          ...   
538                                   0.00                          0.0   
539                                   0.00                          0.0   
540                                   0.00                          0.0   
541                                   0.00                   

In [ ]:
min_gk_games = 6

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[123:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

Zoet, 0.8333333333333334
Pegolo, 0.33333333333333337
Gollini, 0.6666666666666667
Mirante, 0.16666666666666663
Sarr M., 0.33333333333333337
Lamanna, 0.0
Ujkani, 0.16666666666666663
Berisha, 0.0
Marchetti, 0.16666666666666663
Perilli, 0.16666666666666663
Padelli, 0.0
Bardi, 0.16666666666666663
Cordaz, 0.16666666666666663
Pinsoglio, 0.0
Fiorillo, 0.16666666666666663
Cragno, 0.16666666666666663
Sirigu, 0.16666666666666663
Cerofolini, 0.8333333333333334
Rossi F., 0.16666666666666663
Ravaglia F., 0.0
Brancolini, 0.0
Bleve, 0.0
Berardi A., 0.0
Russo A., 0.16666666666666663
Gemello, 0.0
Boer, 0.0
Adamonis, 0.0
Marfella, 0.0
Zovko, 0.16666666666666663
Piana, 0.0
Bagnolini, 0.0
Luis Maximiano, 0.16666666666666663
Svilar, 0.5
Sorrentino A., 0.0
Ciezkowski, 0.0
Saro, 0.0
Vasquez D., 0.0
Turk, 0.6666666666666667


In [ ]:
fc_players = fc_players_newgk

fc_players

id  r                 name       team    surname initial  fb_ID  age  \
0     572  P                Meret     Napoli      Meret            620   25   
1    2814  P             Provedel      Lazio   Provedel            632   28   
2    4964  P              Vicario     Empoli    Vicario            649   25   
3     453  P             Szczesny   Juventus   Szczesny            644   32   
4    2134  P              Falcone      Lecce    Falcone            612   27   
..    ... ..                  ...        ...        ...     ...    ...  ...   
538  5512  A              De Luca  Sampdoria       Luca            151   24   
539  5837  A  Voelkerling Persson      Lecce    Persson            565   19   
540  6113  A            Montevago  Sampdoria  Montevago            365   19   
541  6143  A              Krollis     Spezia    Krollis            288   20   
542  6160  A              Vivaldo    Udinese    Vivaldo             -1    0   

     birth_year  games  ...  gk_pct_goal_kicks_launched  \
0          1997      0  ...                        20.2   
1          1994      0  ...                        33.3   
2          1996      0  ...                        50.0   
3          1990      0  ...                        43.1   
4          1995      0  ...                        79.1   
..          ...    ...  ...                         ...   
538        1998      2  ...                         0.0   
539        2003      9  ...                         0.0   
540        2003      6  ...                         0.0   
541        2001      4  ...                         0.0   
542           0      0  ...                         0.0   

     gk_goal_kick_length_avg  gk_crosses  gk_crosses_stopped  \
0                       26.9       335.0                11.0   
1                       33.2       493.0                21.0   
2                       42.9       606.0                34.0   
3                       38.3       389.0                11.0   
4                       53.5       534.0                23.0   
..                       ...         ...                 ...   
538                      0.0         0.0                 0.0   
539                      0.0         0.0                 0.0   
540                      0.0         0.0                 0.0   
541                      0.0         0.0                 0.0   
542                      0.0         0.0                 0.0   

     gk_crosses_stopped_pct  gk_def_actions_outside_pen_area  \
0                       3.3                             39.0   
1                       4.3                             55.0   
2                       5.6                             22.0   
3                       2.8                             21.0   
4                       4.3                             42.0   
..                      ...                              ...   
538                     0.0                              0.0   
539                     0.0                              0.0   
540                     0.0                              0.0   
541                     0.0                              0.0   
542                     0.0                              0.0   

     gk_def_actions_outside_pen_area_per90  gk_avg_distance_def_actions  \
0                                     1.15                         17.0   
1                                     1.45                         16.5   
2                                     0.71                         11.5   
3                                     0.76                         14.7   
4                                     1.11                         14.1   
..                                     ...                          ...   
538                                   0.00                          0.0   
539                                   0.00                          0.0   
540                                   0.00                          0.0   
541                                   0.00                   

In [ ]:
fc_players.to_excel('mid_outputs/season' + season + '/players_stats.xlsx')